In [29]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import json


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Froms
from src.gcpUtils.auth import getCredentials
from src.gcpUtils.bigQuery import pandasToBq, tableToPandas
from src.gcpUtils.google_storage_manager import *

cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")
# --- 1. Constantes de Negócio ---

# Caminho para seu arquivo de distâncias
ARQUIVO_DISTANCIAS = '../dados/distancias_todas_combinacoes_BIDIRECIONAL.csv'

# Constantes para a fórmula de Leadtime Teórico
K = 6
K_prime = 3
p = 6
q = 4
C = 0.0225
alpha = 2

ARQUIVO_JSON_FILIAIS = '../dados/filiais_inferior_30.json'
DATA_ANALISE = '2025-08-16'  # Formato AAAA-MM-DD
TABELA_BIGQUERY = 'planejamento-animale-292719.checklists_rollout.ANIMALE_checklist'
PATH_CREDENCAIS = '../bd/planejamento-animale-292719-296d49ccdea6.json'


In [30]:

try:
    print(f"Carregando filiais do arquivo: {ARQUIVO_JSON_FILIAIS}...")
    
    with open(ARQUIVO_JSON_FILIAIS, 'r', encoding='utf-8') as f:
        dados_filiais = json.load(f)
        
    df_filiais = pd.DataFrame(dados_filiais)
    
    lista_nomes_filiais = df_filiais['FILIAL'].unique().tolist()
    
    if not lista_nomes_filiais:
        print("Atenção: Nenhuma filial foi encontrada no arquivo JSON.")
    else:
        print(f"Sucesso. {len(lista_nomes_filiais)} filiais únicas encontradas.")

except FileNotFoundError:
    print(f"Erro: Arquivo JSON não encontrado em '{ARQUIVO_JSON_FILIAIS}'")
    lista_nomes_filiais = []
except Exception as e:
    print(f"Erro ao ler o arquivo JSON: {e}")
    lista_nomes_filiais = []


# --- Montar e Executar a Query ---

if lista_nomes_filiais:
    
    # Ex: Transforma ['Filial A', 'Filial B'] em "('Filial A', 'Filial B')"
    filiais_para_query = "','".join(lista_nomes_filiais)
    filiais_para_query = f"('{filiais_para_query}')"

    query = f"""
        SELECT SKU, FILIAL, VELOCIDADE_VENDA, ALVO, PRESENTE, TRANSITO, 
        EST_TOTAL, CONT_RUPTURA, CONT_FALTA, CONT_EXCESSO, VOLUME_EXCESSO, REGULADOR
        FROM {TABELA_BIGQUERY}
        WHERE FILIAL IN {filiais_para_query}
        AND DATA = '{DATA_ANALISE}'
    """
    try:
        # --- Buscar os dados ---
        
        print("Executando consulta no BigQuery...")
        df_resultado = tableToPandas(query, 'planejamento-animale-292719', cred)
        
        print("\n--- Resultado da Consulta ---")
        if df_resultado.empty:
            print("A consulta não retornou dados.")
        else:
            print(df_resultado)

    except Exception as e:
        print(f"\nErro ao executar a consulta no BigQuery: {e}")

else:
    print("\nAnálise não executada pois nenhuma filial foi carregada.")

Carregando filiais do arquivo: ../dados/filiais_inferior_30.json...
Sucesso. 62 filiais únicas encontradas.
Executando consulta no BigQuery...

--- Resultado da Consulta ---
                           SKU                 FILIAL  VELOCIDADE_VENDA  ALVO  \
0        27.03.1447-1511-TAM_1  ANIMALE ALPHAVILLE CM               0.0   1.0   
1       07.20.7684-03032-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
2        11.05.0565-1802-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
3        11.05.0572-1419-TAM_1     ANIMALE ARACAJU CM               0.0   2.0   
4        27.03.1447-1511-TAM_1     ANIMALE ARACAJU CM               0.0   1.0   
...                        ...                    ...               ...   ...   
124313  52.13.6217-03032-TAM_1              BUZIOS CM               0.0   2.0   
124314   52.13.6307-0005-TAM_2              BUZIOS CM               0.0   1.0   
124315   52.13.6379-0005-TAM_4              BUZIOS CM               0.0   1.0   
124316  52.15.16

In [31]:
# ---------------------------------------------------------------------------
# Bloco 1: Funções de Carregamento e Auxiliares
# ---------------------------------------------------------------------------

def carregar_matriz_distancia(caminho_csv):
    """
    Carrega o CSV de distâncias, selecionando apenas as colunas necessárias.
    Retorna um DataFrame vazio em caso de erro.
    """
    colunas_necessarias = ['Filial_A', 'Filial_B', 'Distancia_km']
    try:
        df_dist = pd.read_csv(
            caminho_csv,
            usecols=colunas_necessarias
        )
        print(f"Matriz de distâncias carregada com {len(df_dist)} combinações.")
        return df_dist
    except FileNotFoundError:
        print(f"Erro: Arquivo CSV não encontrado em '{caminho_csv}'")
        return pd.DataFrame(columns=colunas_necessarias)
    except ValueError as e:
        # Erro comum se uma das colunas (usecols) não existir no CSV
        print(f"Erro ao carregar CSV (verifique as colunas): {e}")
        return pd.DataFrame(columns=colunas_necessarias)
    except Exception as e:
        print(f"Erro inesperado ao carregar CSV: {e}")
        return pd.DataFrame(columns=colunas_necessarias)


def _preparar_dados(df_estoque, df_distancias):
    """
    (Auxiliar) Limpa e padroniza os DataFrames de entrada.
    Cria cópias para evitar SettingWithCopyWarning.
    """
    df_estoque_proc = df_estoque.copy()
    df_dist_proc = df_distancias.copy()

    # Padroniza colunas de filiais
    df_estoque_proc['FILIAL'] = df_estoque_proc['FILIAL'].astype(str).str.strip()
    df_dist_proc['Filial_A'] = df_dist_proc['Filial_A'].astype(str).str.strip()
    df_dist_proc['Filial_B'] = df_dist_proc['Filial_B'].astype(str).str.strip()

    # Renomeia colunas de estoque, se necessário
    if 'PRESENTE' in df_estoque_proc.columns:
        df_estoque_proc = df_estoque_proc.rename(columns={'PRESENTE': 'EST_DISP'})

    return df_estoque_proc, df_dist_proc


def _identificar_fontes_destinos(df_estoque):
    """
    (Auxiliar) Calcula a demanda líquida e filtra as lojas-fonte e lojas-destino.
    """
    # --- 1. Calcular Demanda Líquida ---
    # Garante que a demanda nunca seja negativa
    df_estoque['DEMANDA_LIQUIDA'] = np.maximum(
        0,
        df_estoque['ALVO'].fillna(0) - df_estoque['EST_TOTAL'].fillna(0)
    )

    # --- 2. Identificar TODAS as Fontes ---
    df_fontes = df_estoque[
        df_estoque['VOLUME_EXCESSO'].fillna(0) > 0
    ][['SKU', 'FILIAL', 'VOLUME_EXCESSO']].rename(columns={'FILIAL': 'FILIAL_FONTE'})

    # --- 3. Identificar TODOS os Destinos ---
    criterio_demanda = (df_estoque['DEMANDA_LIQUIDA'] > 0)
    criterio_ruptura = (df_estoque['CONT_RUPTURA'] == 1)
    criterio_falta = (df_estoque['CONT_FALTA'] == 1)
    
    df_destinos = df_estoque[criterio_demanda | criterio_ruptura | criterio_falta][[
        'SKU', 'FILIAL', 'DEMANDA_LIQUIDA',
        'CONT_RUPTURA', 'CONT_FALTA', 'VELOCIDADE_VENDA'
    ]].rename(columns={'FILIAL': 'FILIAL_DESTINO'})

    return df_fontes, df_destinos


def _criar_pares_transferencia(df_fontes, df_destinos, df_distancias):
    """
    (Auxiliar) Cria os pares N x N (SKU/Fonte/Destino) e anexa as distâncias.
    """
    # --- 4. Criar Pares de Transferência (N x N) ---
    df_pares = pd.merge(df_fontes, df_destinos, on='SKU')
    
    # Remove transferências para a própria loja
    df_pares = df_pares[df_pares['FILIAL_FONTE'] != df_pares['FILIAL_DESTINO']]
    
    if df_pares.empty:
        print("Nenhum par de transferência válido (Fonte != Destino) foi encontrado.")
        return pd.DataFrame()

    # --- 5. Juntar Distâncias ---
    df_ranking = pd.merge(
        df_pares,
        df_distancias,
        left_on=['FILIAL_FONTE', 'FILIAL_DESTINO'],
        right_on=['Filial_A', 'Filial_B']
    )

    if df_ranking.empty:
        print("Erro: Nenhum par de transferência encontrou uma distância correspondente no CSV.")
        return pd.DataFrame()
        
    print(f"\n--- {len(df_ranking)} Pares de transferência encontrados ---")
    return df_ranking


def _calcular_scores_ranking(df_ranking):
    """
    (Auxiliar) Aplica a fórmula de negócio para calcular os scores de ranking.
    """
    df_calc = df_ranking.copy()

    # --- 6. Cálculo dos Fatores da Fórmula ---
    df_calc['FATOR_PREFERENCIA'] = np.where(
        df_calc['FILIAL_FONTE'] == "CENTRO DE DISTRIBUICAO",
        alpha,
        1
    )

    # Fator Logística
    df_calc['LEADTIME'] = C * df_calc['Distancia_km']
    df_calc['CUSTO_FRETE'] = K * df_calc['Distancia_km']
    # +1 para evitar divisão por zero se distância for 0
    fator_logistica = (1 / (df_calc['LEADTIME'] + 1)) + (1 / (df_calc['CUSTO_FRETE'] + 1))

    # Fator Demanda
    cont_ruptura = df_calc['CONT_RUPTURA'].fillna(0)
    cont_falta = df_calc['CONT_FALTA'].fillna(0)
    fator_demanda = (cont_ruptura * p) + (cont_falta * q)

    # velocidade_venda = df_calc['VELOCIDADE_VENDA'].fillna(0)
    

    # --- 7. Ranking Bruto e Final ---
    df_calc['RANKING_TRANSFERENCIA_BRUTO'] = (
        fator_logistica * fator_demanda * df_calc['FATOR_PREFERENCIA']
    )

    # Quantidade real a ser transferida é o mínimo entre o excesso e a necessidade
    df_calc['QTD_A_TRANSFERIR'] = np.minimum(
        df_calc['VOLUME_EXCESSO'], df_calc['DEMANDA_LIQUIDA']
    )

    # Zera o ranking se não houver o que transferir (sem excesso ou sem demanda)
    df_calc['RANKING_TRANSFERENCIA'] = df_calc['RANKING_TRANSFERENCIA_BRUTO']
    df_calc.loc[df_calc['QTD_A_TRANSFERIR'] == 0, 'RANKING_TRANSFERENCIA'] = 0

    return df_calc


def _finalizar_ranking(df_ranking_calculado):
    """
    (Auxiliar) Ordena, seleciona e limpa as colunas para o resultado final.
    """
    # --- 8. Limpar e Ordenar ---
    df_resultado = df_ranking_calculado.sort_values(by='RANKING_TRANSFERENCIA', ascending=False)

    colunas_finais = [
        'SKU',
        'FILIAL_FONTE',
        'FILIAL_DESTINO',
        'RANKING_TRANSFERENCIA',
        'QTD_A_TRANSFERIR',
        'VELOCIDADE_VENDA',
        'CONT_RUPTURA',
        'CONT_FALTA',
        'VOLUME_EXCESSO',
        'DEMANDA_LIQUIDA',
        'Distancia_km',
        'FATOR_PREFERENCIA',
        'LEADTIME',
        'CUSTO_FRETE'
    ]

    # Garante que apenas colunas existentes sejam selecionadas
    colunas_existentes = [col for col in colunas_finais if col in df_resultado.columns]

    print(f"  Ranking global N x N calculado.")
    transferencias_sugeridas = df_resultado[df_resultado['RANKING_TRANSFERENCIA'] > 0].shape[0]
    print(f"  Total de transferências possíveis sugeridas: {transferencias_sugeridas}")

    return df_resultado[colunas_existentes].reset_index(drop=True)


def encontrar_melhores_fontes_para_sku_destino(df_ranking_global, sku_desejado):
    """
    Para um SKU específico, encontra a *única* melhor filial-fonte
    para *cada* filial-destino que o demanda.

    Garante que cada destino receba de apenas uma fonte (a de maior ranking).
    """
    if df_ranking_global.empty:
        print("DataFrame de ranking global está vazio.")
        return pd.DataFrame()

    sku_desejado_limpo = str(sku_desejado).strip()
    df_sku = df_ranking_global[
        df_ranking_global['SKU'].astype(str).str.strip() == sku_desejado_limpo
    ]

    if df_sku.empty:
        print(f"Nenhuma transferência encontrada para o SKU: {sku_desejado_limpo}")
        return pd.DataFrame(columns=df_ranking_global.columns)

    print(f"\nAnalisando {len(df_sku)} pares possíveis para o SKU {sku_desejado_limpo}...")

    df_best_source_per_dest = df_sku.groupby('FILIAL_DESTINO').first()

    df_best_source_per_dest = df_best_source_per_dest.reset_index()
    df_best_source_per_dest = df_best_source_per_dest.sort_values(
        by='RANKING_TRANSFERENCIA',
        ascending=False
    )

    print(f"Encontradas {len(df_best_source_per_dest)} transferências ótimas "
          f"(uma para cada destino do SKU).")

    return df_best_source_per_dest
# ---------------------------------------------------------------------------
# Bloco 2: Funções Principais de Orquestração e Análise
# ---------------------------------------------------------------------------

def calcular_ranking_global_nxn(df_estoque, df_distancias):
    """
    Orquestra o cálculo completo do ranking N x N de transferências.

    Parâmetros:
    - df_estoque (pd.DataFrame): DataFrame com dados de estoque, demanda, etc.
    - df_distancias (pd.DataFrame): DataFrame com distâncias (Filial_A, Filial_B, Distancia_km).
    - alpha (float): Peso para "CENTRO DE DISTRIBUICAO".
    - C (float): Constante para cálculo de LEADTIME.
    - K (float): Constante para cálculo de CUSTO_FRETE.
    - p (float): Peso para CONT_RUPTURA.
    - q (float): Peso para CONT_FALTA.
    
    Retorna:
    - (pd.DataFrame): DataFrame ordenado com todas as transferências possíveis.
    """
    if df_estoque.empty or df_distancias.empty:
        print("Erro: Dados de estoque ou distâncias estão vazios. Encerrando.")
        return pd.DataFrame()

    # Bloco 1: Preparação
    df_estoque_proc, df_dist_proc = _preparar_dados(df_estoque, df_distancias)

    # Bloco 2: Identificar Atores (Fontes e Destinos)
    df_fontes, df_destinos = _identificar_fontes_destinos(df_estoque_proc)
    
    if df_fontes.empty:
        print("Nenhuma loja-fonte com estoque em excesso foi encontrada.")
        return pd.DataFrame()
    if df_destinos.empty:
        print("Nenhuma loja-destino com demanda encontrada.")
        return pd.DataFrame()

    # Bloco 3: Criar Pares
    df_ranking_inicial = _criar_pares_transferencia(df_fontes, df_destinos, df_dist_proc)
    
    if df_ranking_inicial.empty:
        print("Cálculo interrompido, nenhum par de transferência válido foi encontrado.")
        return pd.DataFrame()

    # Bloco 4: Calcular Scores
    df_ranking_calculado = _calcular_scores_ranking(
        df_ranking_inicial
    )

    # Bloco 5: Finalizar
    df_resultado_final = _finalizar_ranking(df_ranking_calculado)

    return df_resultado_final


def encontrar_melhor_match_para_sku(df_ranking_global, sku_desejado):
    """
    Filtra o ranking global (já ordenado) para encontrar a melhor opção 
    para um SKU específico.

    Parâmetros:
    - df_ranking_global (pd.DataFrame): O DataFrame retornado por `calcular_ranking_global_nxn`.
    - sku_desejado (str/int): O SKU a ser buscado.

    Retorna:
    - (pd.DataFrame): Um DataFrame com a melhor linha (top 1) para o SKU, 
                      ou um DataFrame vazio se o SKU não for encontrado.
    """
    if df_ranking_global.empty:
        print("DataFrame de ranking global está vazio.")
        return pd.DataFrame()

    sku_desejado_limpo = str(sku_desejado).strip()
    
    # Filtra pelo SKU. Como o DF já está ordenado, .head(1) pega o melhor.
    df_sku = df_ranking_global[
        df_ranking_global['SKU'].astype(str).str.strip() == sku_desejado_limpo
    ]

    if df_sku.empty:
        print(f"Nenhuma transferência encontrada para o SKU: {sku_desejado_limpo}")
        return pd.DataFrame(columns=df_ranking_global.columns)

    return df_sku.head(1)


def encontrar_melhor_match_para_todos_skus(df_ranking_global):
    """
    Agrupa o ranking global para encontrar a *única* melhor transferência 
    (maior ranking) para *cada* SKU.

    Parâmetros:
    - df_ranking_global (pd.DataFrame): O DataFrame retornado por `calcular_ranking_global_nxn`.

    Retorna:
    - (pd.DataFrame): Um DataFrame onde cada linha é o melhor match para um SKU.
    """
    if df_ranking_global.empty:
        print("DataFrame de ranking global está vazio.")
        return pd.DataFrame()

    print(f"\nEncontrando o melhor match para {df_ranking_global['SKU'].nunique()} SKUs únicos...")

    # Como o df_ranking_global JÁ ESTÁ ORDENADO pelo ranking,
    # .first() pega a linha com o maior ranking para cada grupo de SKU.
    df_best_matches = df_ranking_global.groupby('SKU').first()

    # O groupby torna 'SKU' o índice. Vamos resetar para ser uma coluna.
    df_best_matches = df_best_matches.reset_index()

    # Re-ordenar a lista final pela prioridade geral
    df_best_matches = df_best_matches.sort_values(by='RANKING_TRANSFERENCIA', ascending=False)

    print(f"Melhores matches encontrados.")
    return df_best_matches

In [32]:
if __name__ == "__main__":
    
    
    print("--- ETAPA 1: Preparando Dados (Lojas + CD) ---")
    
    df_distancias_total = carregar_matriz_distancia(ARQUIVO_DISTANCIAS)
    df_estoque_dia = df_resultado.copy()

    # Limpa a coluna FILIAL original
    df_estoque_dia['FILIAL'] = df_estoque_dia['FILIAL'].astype(str).str.strip()

    df_cd_estoque = df_estoque_dia.groupby('SKU', as_index=False)['REGULADOR'].first()
    df_cd_estoque['FILIAL'] = "CENTRO DE DISTRIBUICAO".strip()
    df_cd_estoque = df_cd_estoque.rename(columns={'REGULADOR': 'VOLUME_EXCESSO'})
    
    df_estoque_completo = pd.concat([df_estoque_dia, df_cd_estoque], ignore_index=True)
    
    print("\n--- ETAPA 2: Calculando Ranking Global N-para-N ---")

    df_ranking_global = calcular_ranking_global_nxn(
        df_estoque_completo, 
        df_distancias_total,
    )
    
    if not df_ranking_global.empty:
        print("\n--- ETAPA 3: Testando a Função 1 (Match para SKU específico) ---")
        sku = "25.34.2744-0101-TAM_2"
        
        df_melhor_match_sku = encontrar_melhor_match_para_sku(df_ranking_global, sku)
        
        if not df_melhor_match_sku.empty:
            print(f"Melhor match encontrado para {sku}:")
            print(df_melhor_match_sku.to_markdown(index=False, floatfmt=".4f"))
        else:
            print(f"Nenhum match encontrado para {sku}")
        
        print("\n--- ETAPA 4: Testando a Função 2 (Melhor match para TODOS os SKUs) ---")
        df_melhores_matches_todos_skus = encontrar_melhor_match_para_todos_skus(df_ranking_global)
        
        print("Melhores matches encontrados (um por SKU):")
        print(df_melhores_matches_todos_skus.to_markdown(index=False, floatfmt=".4f"))

        print(f"\n--- ETAPA 5: Testando a Função 3 (Melhores fontes p/ destinos do SKU: {sku}) ---")
        df_melhores_fontes_sku = encontrar_melhores_fontes_para_sku_destino(df_ranking_global, sku)

        if not df_melhores_fontes_sku.empty:
            print(f"Melhores fontes para CADA destino do SKU {sku}:")
            print(df_melhores_fontes_sku.head().to_markdown(index=False, floatfmt=".4f"))
            # salva em excel
            df_melhores_fontes_sku.to_excel(f"melhores_fontes_sku_{sku}.xlsx", index=False)
        else:
            print(f"Nenhuma fonte/destino encontrado para {sku}")

    else:
        print("\nCálculo do ranking N-para-N não gerou resultados. Funções de match não executadas.")

--- ETAPA 1: Preparando Dados (Lojas + CD) ---
Matriz de distâncias carregada com 3906 combinações.

--- ETAPA 2: Calculando Ranking Global N-para-N ---

--- 129980 Pares de transferência encontrados ---
  Ranking global N x N calculado.
  Total de transferências possíveis sugeridas: 93691

--- ETAPA 3: Testando a Função 1 (Match para SKU específico) ---
Melhor match encontrado para 25.34.2744-0101-TAM_2:
| SKU                   | FILIAL_FONTE           | FILIAL_DESTINO         |   RANKING_TRANSFERENCIA |   QTD_A_TRANSFERIR |   VELOCIDADE_VENDA |   CONT_RUPTURA |   CONT_FALTA |   VOLUME_EXCESSO |   DEMANDA_LIQUIDA |   Distancia_km |   FATOR_PREFERENCIA |   LEADTIME |   CUSTO_FRETE |
|:----------------------|:-----------------------|:-----------------------|------------------------:|-------------------:|-------------------:|---------------:|-------------:|-----------------:|------------------:|---------------:|--------------------:|-----------:|--------------:|
| 25.34.2744-0101-TAM_2 |

In [33]:
def exibir_debug_sku(sku_desejado, df_estoque_completo, df_ranking_global):
    """
    Exibe um relatório de debug detalhado para um único SKU,
    mostrando fontes, destinos e todos os parâmetros de cálculo.
    """
    
    print("="*60)
    print(f"INICIANDO DEBUG PARA O SKU: {sku_desejado}")
    print("="*60)
    
    # --- 1. Limpeza e Validação ---
    sku_desejado = str(sku_desejado).strip()
    
    df_sku_data = df_estoque_completo[
        df_estoque_completo['SKU'].astype(str).str.strip() == sku_desejado
    ].copy()
    
    if df_sku_data.empty:
        print(f"ERRO: SKU '{sku_desejado}' não foi encontrado no DataFrame de estoque.")
        return

    # --- 2. Exibir Fontes (Quem TEM o SKU) ---
    print("\n## 1. FONTES (Quem tem este SKU)")
    df_fontes = df_sku_data[df_sku_data['VOLUME_EXCESSO'].fillna(0) > 0]
    
    if df_fontes.empty:
        print("Nenhuma filial fonte encontrada com excesso/estoque deste SKU.")
    else:
        print(df_fontes[['FILIAL', 'VOLUME_EXCESSO']].to_markdown(index=False))

    # --- 3. Exibir Destinos (Quem PRECISA do SKU) ---
    print("\n## 2. DESTINOS (Quem precisa deste SKU)")
    
    if 'DEMANDA_LIQUIDA' not in df_sku_data.columns:
        print("  (Calculando 'DEMANDA_LIQUIDA' para o debug...)")
        df_sku_data['DEMANDA_LIQUIDA'] = np.maximum(0, df_sku_data['ALVO'].fillna(0) - df_sku_data['EST_TOTAL'].fillna(0))

    df_destinos = df_sku_data[
        (df_sku_data['DEMANDA_LIQUIDA'].fillna(0) > 0) | 
        (df_sku_data['CONT_RUPTURA'].fillna(0) == 1)
    ]
    
    if df_destinos.empty:
        print("Nenhuma filial destino encontrada com demanda para este SKU.")
    else:
        cols_destino = ['FILIAL', 'DEMANDA_LIQUIDA', 'CONT_RUPTURA', 'CONT_FALTA', 'VELOCIDADE_VENDA']
        cols_destino_existentes = [col for col in cols_destino if col in df_destinos.columns]
        print(df_destinos[cols_destino_existentes].to_markdown(index=False, floatfmt=".2f"))

    # --- 4. Exibir Cálculos (O "Matching" N-para-N) ---
    print("\n## 3. CÁLCULO DE RANKING (Todos os Matches Possíveis)")
    
    # Filtra o ranking global (que agora tem 'VELOCIDADE_VENDA')
    df_ranking_sku = df_ranking_global[
        df_ranking_global['SKU'].astype(str).str.strip() == sku_desejado
    ]
    
    if df_ranking_sku.empty:
        print("Nenhum match (Fonte -> Destino) foi encontrado ou calculado para este SKU.")
    else:
        print("Mostrando todos os parâmetros calculados para cada combinação:")
        print(df_ranking_sku.to_markdown(index=False, floatfmt=".4f"))
        
        print("\n---")
        print("MELHOR MATCH ENCONTRADO:")
        print(df_ranking_sku.head(1).to_markdown(index=False, floatfmt=".4f"))
        
    print("="*60)
    print(f"FIM DO DEBUG PARA O SKU: {sku_desejado}")
    print("="*60)
    
    # Salva o arquivo de debug específico do SKU
    try:
        nome_debug_excel = f"debug_sku_{sku_desejado}.xlsx"
        df_ranking_sku.to_excel(nome_debug_excel, index=False)
        print(f"Arquivo de debug salvo em: '{nome_debug_excel}'")
    except Exception as e:
        print(f"Erro ao salvar o arquivo de debug: {e}")
    

    # --- 4. Execução Principal ---
# --- 4. Execução Principal ---
if __name__ == "__main__":
    
    df_estoque_dia = df_resultado.copy()
    df_distancias_total = carregar_matriz_distancia(ARQUIVO_DISTANCIAS)

    df_estoque_dia['FILIAL'] = df_estoque_dia['FILIAL'].astype(str).str.strip()

    if 'REGULADOR' in df_estoque_dia.columns:
        df_cd_estoque = df_estoque_dia.groupby('SKU', as_index=False)['REGULADOR'].first()
        df_cd_estoque['FILIAL'] = "CENTRO DE DISTRIBUICAO".strip()
        df_cd_estoque = df_cd_estoque.rename(columns={'REGULADOR': 'VOLUME_EXCESSO'})
        
        df_estoque_completo = pd.concat([df_estoque_dia, df_cd_estoque], ignore_index=True)
    else:
        print("Aviso: Coluna 'REGULADOR' não encontrada. O CD não será criado como fonte.")
        df_estoque_completo = df_estoque_dia.copy()

    
    df_ranking_global = calcular_ranking_global_nxn(
        df_estoque_completo, 
        df_distancias_total,
    )
    
    if not df_ranking_global.empty:
        
        exibir_debug_sku(
            sku_desejado="25.34.2744-0101-TAM_2", # <-- O SKU que você escolheu
            df_estoque_completo=df_estoque_completo,
            df_ranking_global=df_ranking_global
        )
        
        df_melhores_matches = encontrar_melhor_match_para_todos_skus(df_ranking_global)
        nome_arquivo = "ranking_transferencias.xlsx"
        
        try:
            with pd.ExcelWriter(nome_arquivo, engine='openpyxl') as writer:
                df_ranking_global.to_excel(
                    writer, 
                    sheet_name='Ranking_Completo_NxN', 
                    index=False
                )
                
                df_melhores_matches.to_excel(
                    writer, 
                    sheet_name='Melhor_Match_por_SKU', 
                    index=False
                )
            
            print(f"Sucesso! Resultados GLOBAIS salvos em '{nome_arquivo}' com 2 abas.")

        except Exception as e:
            print(f"Erro ao salvar o arquivo Excel global: {e}")

    else:
        print("\nCálculo do ranking N-para-N não gerou resultados. Nada foi salvo.")

Matriz de distâncias carregada com 3906 combinações.

--- 129980 Pares de transferência encontrados ---
  Ranking global N x N calculado.
  Total de transferências possíveis sugeridas: 93691
INICIANDO DEBUG PARA O SKU: 25.34.2744-0101-TAM_2

## 1. FONTES (Quem tem este SKU)
| FILIAL                   |   VOLUME_EXCESSO |
|:-------------------------|-----------------:|
| ANIMALE ALPHAVILLE CM    |                1 |
| ANIMALE BH SHOP CM       |                1 |
| ANIMALE CAMPINAS CM      |                1 |
| ANIMALE FLORIANOPOLIS CM |                1 |
| ANIMALE IGUATEMI BSB CM  |                1 |
| ANIMALE MOEMA CM         |                1 |
| ANIMALE MORUMBI CM       |                1 |
| ANIMALE PORTO ALEGRE CM  |                1 |
| ANIMALE RIO DESIGN CM    |                1 |
| ANIMALE SHOP BATEL CM    |                1 |
| CENTRO DE DISTRIBUICAO   |                1 |

## 2. DESTINOS (Quem precisa deste SKU)
  (Calculando 'DEMANDA_LIQUIDA' para o debug...)
| FILIAL   